# Step1 기초 작업

In [68]:
import pandas as pd
import os
import sys
from selenium import webdriver as wd
import time
import random # 엑셀을 다운받는 행위에 대한 시간 간격을 변조
import selenium
from selenium.webdriver.common.by import By


selenium.__version__

'4.11.2'

In [69]:
# 사이트 : https://www.opinet.co.kr/searRgSelect.do
# 전국 주유소 가격 정보(일단위) 수집 : 24:00 수집 목표 (1일 1회) -> 변동시간 확인 후 추가 설정
# 시도 > 시군구 > 엑셀다운로드 -> *.XLSX 다운(윈도우 다운로드 폴더) -> 다 모아서 DF로 구성(pandas) -> 엑셀 삭제(OS 모듈) -> db로


In [70]:
# 브라우저 가동
driver = wd.Chrome()

In [71]:
# 초기 페이지 접속
target_site = 'https://www.opinet.co.kr/searRgSelect.do#'
driver.get(target_site)
time.sleep(2) # 사이트가 늦게 뜨면 대기, 디도스 방어코드 (cloudflare => 10초)


# STEP 2 지역 정보 획득
- 지역 정보 획득 - select 태그

In [72]:
# 좌측 지역 (서울...) 등 지역(17<지역> + 1<프럼프트>)이 담겨 있는 태그 => select#SIDO_NM0 > option
sidos = driver.find_elements(By.CSS_SELECTOR, 'select#SIDO_NM0 > option')
len(sidos)

18

In [73]:
for sido in sidos[1:] : # 프람프트 제외
    value = sido.get_attribute('value').strip()
    print(value, end = ', ')

서울특별시, 부산광역시, 대구광역시, 인천광역시, 광주광역시, 대전광역시, 울산광역시, 세종특별자치시, 경기도, 충청북도, 충청남도, 전라북도, 전라남도, 경상북도, 경상남도, 제주특별자치도, 강원특별자치도, 

In [74]:
sido_values = [sido.get_attribute('value').strip() for sido in sidos[1:]]
print(sido_values) # 최상위 카테고리

['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시', '세종특별자치시', '경기도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도', '경상남도', '제주특별자치도', '강원특별자치도']


In [75]:
# 사용자가 경기를 클릭했다 -> select 태그를 특정해서, 부산값을 넣고, send_keys() 하는 방식으로 이해할 수 있음.
# 자동 위치 완성때문에 다른 지역으로 가는듯함
SIDO_NM0_SEL = driver.find_element(By.ID, 'SIDO_NM0')
# 값 입력 -> 샘플로 부산(1)
SIDO_NM0_SEL.send_keys(sido_values[2])

In [76]:
sido_values

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도',
 '강원특별자치도']

In [77]:
# 연습 -> 순환(서울~강원) 로테이션 진행, 1초 대기 -> 정말 지역만 바뀜! 주유소 목록은 바뀌지 않음.
for value in sido_values[:1]:
    # value : 서울특별시, 부산광역시, ...
    # 지역 변경
    SIDO_NM0_SEL = driver.find_element(By.ID, 'SIDO_NM0')
    SIDO_NM0_SEL.send_keys(value)
    time.sleep(1)
# 이제 시/군/구를 찍어봐야겠다!

In [78]:
# 시/군/구 선택(for) -> 잠시 대기 -> 엑셀 저장 클릭
# 시/군/구 값 획득
# 시군구 select 요소 id = SIGUNGU_NM0
# 시군구 select 요소 id 밑 모든 option = SIGUNGU_NM0 > option
sigungus = driver.find_elements(By.CSS_SELECTOR, '#SIGUNGU_NM0 > option')
len(sigungus) # 프럼프트 1 + 나머지
[opt.get_attribute('value') for opt in sigungus[1:]]


['강남구',
 '강동구',
 '강북구',
 '강서구',
 '관악구',
 '광진구',
 '구로구',
 '금천구',
 '노원구',
 '도봉구',
 '동대문구',
 '동작구',
 '마포구',
 '서대문구',
 '서초구',
 '성동구',
 '성북구',
 '송파구',
 '양천구',
 '영등포구',
 '용산구',
 '은평구',
 '종로구',
 '중구',
 '중랑구']

In [81]:
# 시군구값 획득

# 1. 시군구 값 획득
sigungus = driver.find_elements(By.CSS_SELECTOR, '#SIGUNGU_NM0 > option')
sigungus = [opt.get_attribute('value') for opt in sigungus[1:]]

# sigungus : ['강남구', '강동구' ... ]
for sigungu in sigungus[:3] :
    # 2. 시군구 요소 획득 -> 시군구가 선택되면 메모리가 변경됨 (화면갱신)
    tmp = driver.find_element(By.CSS_SELECTOR, '#SIGUNGU_NM0')
    
    # 3. 시군구 선택(하나씩) -> form전송(화면 반짝, X자 발견) -> 화면 갱신
    tmp.send_keys(sigungu)

    # 4. 잠시 대기 (화면 껌뻑, 새로 화면이 모두 랜더링이 된다)
    time.sleep(1*5)

    # 5. 엑셀 저장 버튼 클릭
    driver.find_element(By.ID, 'glopopd_excel').click()

In [82]:
# 최종코드
# 최종코드 
# step 1
target_site = 'https://www.opinet.co.kr/searRgSelect.do'
driver.get( target_site )
time.sleep(2) # 사이트가 늦게 뜨면 대기, 디도그 방어코드 (cloudflare => 10초)

# step 2
sidos = driver.find_elements(By.CSS_SELECTOR, 'select#SIDO_NM0 > option')
sido_values = [ sido.get_attribute('value').strip() for sido in sidos[1:] ]

# step 3
for value in sido_values[:1]:#sidos[1:]:
    # value : 서울특별시 -> 부산광역시 -> ...
    # 지역변경 
    # 하위 시군구에서 화면이 갱신되기 때문에 매번 새로 구한다
    SIDO_NM0_SEL = driver.find_element(By.ID,'SIDO_NM0')
    SIDO_NM0_SEL.send_keys( value )
    # 1초 대기
    time.sleep(1)
    
    # 시군구 값 획득 -> 시군구 선택(for) -> 잠시대기 -> 엑셀저장 클릭
    # 1. 시군구 값 획득
    sigungus = driver.find_elements(By.CSS_SELECTOR, '#SIGUNGU_NM0 > option')
    sigungus = [ opt.get_attribute('value') for opt in sigungus[1:] ]
    #print( sigungus )

    for sigungu in sigungus:#[:3]: # ['강남구', '강동구', ...]
        # 2. 시군구 요소 획득 -> 시군구가 선택되면 메모리가 변경됨(화면갱신)
        tmp = driver.find_element(By.CSS_SELECTOR, '#SIGUNGU_NM0')

        # 3. 시군구 선택(햐나씩) -> form 전송 발생 : 화면 껌벅 : X자 발견 -> 화면갱신
        #    이 결과로 새로운 지역의 주유소 정보가 내려온다
        tmp.send_keys( sigungu )

        # 4. 잠시 대기 (화면껌벅, 새로 화면이 모드 렌더링이 된다)
        time.sleep(1*5)

        # 5. 엑셀 저장 버튼 클릭
        driver.find_element(By.ID, 'glopopd_excel').click()

# Step3 : 엑셀파일 모아서 DF로 구성


In [87]:
# 특정 폴더 내에 필요로 하는 파일들이 존재한다.
# 이 파일들을 모아 하나의 목록(리스트)으로 구성 -> glob
import glob

files = glob.glob('C://Users//NT551XCJ//Downloads//*.xls')
len(files), files[0]

(25, 'C://Users//NT551XCJ//Downloads\\지역_위치별(주유소) (1).xls')

In [106]:
# 요구사항
# 25개의 xls => 25개의 df => 리스트에 담았다 => 1개의 df로 통합(병합) -> db에 1번 입력
import pandas as pd

dfs = list()
for file in files:
    df = pd.read_excel(file, header = 2) # header는 위에서 부터 행
    print(type(df))
    # 리스트의 맴버가 DataFrame이 된다.
    dfs.append(df)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [107]:
# 1개의 df로 병합
all_df = pd.concat(dfs)
all_df.shape
# 서울시 주요소의 개수는 441개

(441, 10)

In [108]:
# 최종 통합된 데이터
all_df.head()

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유
0,서울특별시,재건에너지 재정제2주유소 고속셀프지점,서울특별시 강동구 천호대로 1246 (둔촌제2동),HD현대오일뱅크,02-487-2030,Y,-,1719,1579,-
1,서울특별시,구천면주유소,서울 강동구 구천면로 357 (암사동),HD현대오일뱅크,02-441-0536,N,-,1734,1615,-
2,서울특별시,(주)삼표에너지 고덕주유소,서울 강동구 고덕로 39 (암사동),GS칼텍스,02-441-3327,Y,1945,1745,1625,1635
3,서울특별시,(주)소모 신월주유소,서울 강동구 양재대로 1323 (성내동),GS칼텍스,02-6956-6674,Y,1946,1746,1638,1590
4,서울특별시,명일주유소,서울 강동구 고덕로 168 (명일동),SK에너지,02-3428-1739,Y,1995,1748,1625,-


In [ ]:
# 사용자로부터 입력을 받아서 서버로 전송하는 태그들
'''
    <input name='xxx'
    <select
    <radio
    <checkbox
    <textarea

    xxx가 키가 되고 사용자가 입력(선택,체크)한 값 value가 된다

    selenium에서 값을 세팅하는 방법 => send_keys('값')
'''

# 번외 : Naver 로그인

In [18]:
'''
driver.get('https://nid.naver.com/nidlogin.login?mode=form&url=https://www.naver.com/')
# 아이디 입력창 획득
id_input = driver.find_element(By.ID, 'id')
# 아이디 입력!
id_input.send_keys('t-academy')
# PW 입력창 획득
pw_input = driver.find_element(By.ID, 'pw')
# PW 입력!
pw_input.send_keys('12345678')
# 로그인
driver.find_element(By.ID, "log.login").click() # 매크로로 감지해버림 -> 비정상 처리
# 만약 이용하려면 마우스 위치까지 조정해야함!
'''

In [19]:
# 최종 마무리 작업
# 웹드라이버 반드시 종료, 제거 => 메모리 누적이 안됨
driver.close()
driver.quit()

# s/w 레벨로 자동화 수행
import sys
sys.exit(0)